# CAPTAIN tutorial – Application of optimized policies

After optimizing a policy through the `train_model()` function, the parameters of the policy can be used to perform conservation planning on simulated and empirical datasets.

## Testing a policy on simulated datasets
We can perform conservation planning experiments on simulated datasets to compare the performance of optimized policies, e.g., based on different monitoring strategies, and to assess the trade-offs linked to different policy objectives.
This involves specifying the type of model (monitoring strategy, policy objective) and the output file from the model optimization (trained under the same model settings).
Pre-trained models under different settings can be [downloaded here](https://zenodo.org/api/files/c4663248-5dc2-4932-9d43-ec0c490595ae/trained_models.zip).

To test a policy on simulated datasets we call the `run_policy()` function. This uses several of the arguments also implemented when training a model, such as `obsMode` and `observePolicy` defining the type and frequency of monitoring and the `rewardMode` defining the policy objective (see [Conservation policy optimization tutorial for more details](https://github.com/captain-project/notebooks/blob/main/Tutorial_policy_optimization.ipynb)).
As with the `train_model()` function, the size of the protection units is user-defined, with e.g.,  `resolution = [5,5]` setting protection units of 5 x 5 cells (the default). The number of steps during which the system is monitored without yet establishing protected units is controlled by the parameter `start_protecting` (by default set to 3).

To generate robust statistical expectations under a given policy, we can run it over a number of simulated systems set by the argument `simulations`.
By specifying a seed for the random number generators using the argument `rnd_seed` we ensure that the same sequence of simulations can be used multiple times. This is important to ensure reproducibility of the experiments and to compare the performance of different policies.


### Testing a random policy
To determine a baseline outcome for a given system and budget, we can test a random policy, where protected units are randomly selected within the budget constraints. Among the parameters we can set here are the number of simulations (each will be based on a different system randomly selected from the directory with the pre-simulated datasets), the budget (as a fraction of the total area that can be protected), and the number of steps each simulation will be run for.

In [ ]:
import captain as cn

path_to_sim_data = 'data_dependencies/pickles'

cn.run_policy(simulations=5,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              trained_model=None,
              observePolicy=1,
              wd=path_to_sim_data,  # directory with pre-simulated systems
              outfile="sim_random_policy.log")

Setting `observePolicy=1` implies a dynamic protection strategy where one protection unit is established at each time unit until the budget allows. Alternatively, setting `observePolicy=2` specifies that all protection units will be placed in one step.

### Applying a trained policy

When applying a pre-trained policy, we need to specify 1) the path to the log file generated through model optimization (e.g., variable `trained_model_file` below), 2) the architecture of the neural network (`n_nodes`), and 3) the parameters defining monitoring strategy and policy objective.
The examples below are based on the file names and settings of the pre-trained models [available here](https://zenodo.org/api/files/c4663248-5dc2-4932-9d43-ec0c490595ae/trained_models.zip).

The code below shows how to run 5 simulations under a policy using full recurrent monitoring and optimized to minimize species loss.

In [ ]:
path_to_sim_data = 'data_dependencies/pickles'
trained_model_file = 'trained_models/full_species_monitoring_n4-2.log'

cn.run_policy(simulations=5,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              n_nodes=[4, 2],
              trained_model=trained_model_file,
              obsMode=1, # full species monitoring
              observePolicy=1, # recurrent monitoring, dynamic protection
              rewardMode=0, # objective: minimize species loss
              wd=path_to_sim_data,  # directory with pre-simulated systems
              outfile="citizen-science_recurrent_monitoring.log")

A model optimize to use citizen-science monitoring to minimize biodiversity loss can run using:


In [ ]:
trained_model_file = 'trained_models/citizen_science_d4_n4-2.log'

cn.run_policy(simulations=5,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              n_nodes=[4, 2],
              trained_model=trained_model_file,
              obsMode=2, # citizen-science species monitoring
              observePolicy=1, # recurrent monitoring, dynamic protection
              rewardMode=0, # objective: minimize species loss
              outfile = "sim_citizen-science_recurrent_monitoring.log",
              wd = path_to_sim_data) # directory with pre-simulated systems

while a model trained to monitor the system for the first few steps before spending all the budget in one step is run using the following command (model settings based on those used in the pre-trained model):

In [ ]:
trained_model_file = 'trained_models/full_monitor_protect_at_once_d4_n8-0.log'

cn.run_policy(simulations=5,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              n_nodes=[8,0],
              trained_model=trained_model_file,
              obsMode=1, # full species monitoring
              observePolicy=2, # recurrent monitoring, at-once protection
              rewardMode=0, # objective: minimize species loss
              outfile = "sim_full_monitor_protect_at_once.log",
              wd = path_to_sim_data) # directory with pre-simulated systems

The functions below show how to apply a policy aiming to minimize the loss of economic value and to maximize the amount of protected area, respectively:


In [ ]:
trained_model_file = 'trained_models/value_d4_n4-2.log'

cn.run_policy(simulations=5,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              n_nodes=[8,0],
              trained_model=trained_model_file,
              obsMode=4, # value monitoring
              observePolicy=1, # recurrent monitoring, at-once protection
              rewardMode=1, # objective: minimize value loss
              outfile = "sim_protect_value.log",
              wd = path_to_sim_data) # directory with pre-simulated systems

In [ ]:
trained_model_file = 'trained_models/area_d4_n4-0.log'

cn.run_policy(simulations=5,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              n_nodes=[4,0],
              trained_model=trained_model_file,
              obsMode=5, # area-cost monitoring
              observePolicy=1, # recurrent monitoring, dynamic protection
              rewardMode=2, # objective: maximize protected area
              outfile = "sim_protect_area.log",
              wd = path_to_sim_data) # directory with pre-simulated systems

### Simulations output and plots

The output of simulations run using the `run_policy()` function can be summarized in different ways.
By default, two log files are produced with summary statistics about the simulation outcome:
1. the first is named as defined in the `outfile` argument and is a tab-separaeted table with one row for each simulation providing information of the system in the last step of its evolution. The information includes: simulation number, remaining budget, number pof protected cells, species loss, value loss and phylogenetic diversity loss.
2. The second file, named like `outfile + _step.log` includes similar information but provided for each step of each simulation.

Additionally, a pickle file storing the full simulated system can be saved at each step of each simulation using the argument `save_pkls=True`. These files can can be reloaded in Python to extract detailed information about the system adn its evolution. We note that saving this output can increase the run time of the simulation and the pickle files might take up substantial disk space.

Finally, the simulated systems can be plotted during the run using the argument `plot_sim=True` (example shown below). This will produce one PDF file for each step of each simulation. By deafult the PDF does not include plots species-specific ranges (as this would slow down significantly the plotting function for systems with e.g. hundreds of species). However, with the argument `plot_species` we can specify the indices of the species for which geographic range an abundance should be plotted. For instance, using `plot_species=[0, 100, 200, 300]` we will include range maps of four species with indices 0, 100, 200, 300. Similarly, using `plot_species=range(10)` will result range plots of the first 10 species.
Since the plotting option will result in many PDF files and will increase run time per simulation, it is recommended for visualization but not for extensive simulations.

In [ ]:
path_to_sim_data = 'data_dependencies/pickles'
trained_model_file = 'trained_models/full_species_monitoring_n4-2.log'

cn.run_policy(simulations=1,
              rnd_seed=123,
              budget=0.11,
              steps=20,
              n_nodes=[4, 2],
              trained_model=trained_model_file,
              obsMode=1, # full species monitoring
              observePolicy=1, # recurrent monitoring, dynamic protection
              rewardMode=0, # objective: minimize species loss
              wd=path_to_sim_data,  # directory with pre-simulated systems
              plot_sim=True,
              plot_species=[0,100,300,450],
              outfile="citizen-science_recurrent_monitoring.log")